Let $p = 176027859962212193686395560963$.  I think that $p$ is probably prime. 


# Problem 1

Verify that 6 is a primitive root modulo $p$, but that $2,3,4$ and $5$ are not.

In [5]:
# This function takes in a number and gives the prime factors of that number. A more detailed explanation of the
# process is given in the next problem.
def primer(p):
    factors = []
    index = 1
    while p != 1:
        index += 1
        if p%index == 0:
            truth = 1
            for factor in factors:
                if index%factor==0:
                    truth = 0
            if truth:
                factors.append(index)
                while p%index == 0:
                    p//=index
    return factors
# This is the exact same fast powering algorithm as used in the last assignment.
def faster(a, p, mod):
    square = a
    answer = 1
    while p > 0:
        if p % 2 == 1:
            p -= 1
            answer *= square
            answer %= mod
        p //= 2
        square = square ** 2 % mod
    return answer % mod
# This function calculates the primitive roots of a large prime given a list of potential roots
def primitive(roots, factors, phi, mod):
    # Creating an array to store all found primitive roots
    primitives = []
    # Iterating over the roots provided
    for root in roots:
        # Declaring a truth variable to determine whether the root is a primitive
        truth = True
        # Iterating over the prime factors of our phi
        for factor in factors:
            # Computing root**(phi/factor) mod p
            congruency = faster(root, (phi//factor), mod)
            # If this number is 1, we know the root is not primitive, therefore we set truth to false and break
            if congruency==1:
                truth = False
                break
        # If the truth value remains true, we know this number is a primitive root and therefore append the root to
        # primitives.
        if truth:
            primitives.append(root)
    return primitives
x = 176027859962212193686395560963
x1 = x-1
factors = primer(x1)
roots = [6, 2, 3, 4, 5]
primitives = primitive(roots, factors, x1, x)
print("The primitive roots of " + str(x) + " from the list given are:")
for i in primitives:
    print(i)

The primitive roots of 176027859962212193686395560963 from the list given are:
6


# Problem 2

Alice posts the ElGamal public key $A=20800698928339426549629799773$, for use with the prime $p$ above and the primitive root $g=6$.  Bob sends her the encrypted message

$$c_1 = 95227620837509305858765921398, \quad c_2 = 77039087099389375582873761352.$$

You are Eve - and you see that Alice has made a serious error.  What was Alice's private key, and what was Bob's message?


In [2]:
import math as math
# This function is the same gcdex function as implemented on the last assignment
# Instead of returning a tuple containing the two coefficients and the gcd, I modified it to only return the
# coefficient of the term that is not p.
def gcdex(a, b):
    p1 = 1
    q1 = 0
    h1 = a
    p2 = 0
    q2 = 1
    h2 = b
    while h2 != 0:
        r = h1 // h2
        p3 = p1 - r * p2
        q3 = q1 - r * q2
        h3 = h1 - r * h2
        p1 = p2
        q1 = q2
        h1 = h2
        p2 = p3
        q2 = q3
        h2 = h3
    return p1
# This is exactly the same fast powering algorithm as was implemented in the previous assignment
def faster(a, p, mod):
    square = a
    answer = 1
    while p > 0:
        if p % 2 == 1:
            p -= 1
            answer *= square
            answer %= mod
        p //= 2
        square = square ** 2 % mod
    return answer % mod
# This function factors p into only small prime factors. Normally, with a more difficult elgamal problem, we would need
# to keep track of the multiplicity of prime but in this assignment, we know each of the primes have a multiplicity of
# 1. 
def primes(p):
    # Creating an array to store all of the factors
    factors = []
    # Creating an index to keep track of the number we are seeing if divides p
    index = 1
    while p != 1:
        index += 1
        # If the number we are currently on divides p ->
        if p%index == 0:
            # Setting our truth variable to true
            truth = 1
            # We first check if this number is divisible by any of the factors already in our list, meaning it would
            # be a multiple of prime factors and not a prime factor itself.
            # If this occurs, our truth variable is set to false.
            for factor in factors:
                if index%factor==0:
                    truth = 0
            # If our truth variable is still true, we p by our number until the remainder of p/index is not 0.
            # We then add this prime factor to our list of prime factors
            # As priorly stated, in a more complicated problem we would need to keep track of multiplicity but that
            # does not apply here.
            if truth:
                while p%index == 0:
                    p//=index
                factors.append(index)
    # Returning our list of prime factors
    return factors

def pohlig(g, A, mod):
    # Computing phi which we know is p-1 as p is prime.
    phi = mod - 1
    # Factoring phi into its prime factors
    prime_factors = primes(phi)
    # Creating an array for all x_0
    x_naughts = []
    # Iterating through the prime factors
    for prime in prime_factors:
        # Computing the exponent to be phi divided by the prime factor
        exp = phi//prime
        # Using the fast powering algorithm to raise both g and A to the power of the exp mod p
        A_exp = faster(A, exp, mod)
        g_exp = faster(g, exp, mod)
        # Finding the x_0 by raising g**(phi/prime) to a power less than the prime factor and checking if it is 
        # congrunt to A**(phi/prime) mod p
        # If it is, append it to the list of x_0 and break from the for loop to check the next prime
        for i in range(prime):
            if faster(g_exp, i, mod)==A_exp:
                x_naughts.append(i)
                break
    # Setting the value to sum the results of the Chinese Remainder Theorem to 0
    unmod = 0
    # Iterating of the length of x_naughts so that the index will be the same as in prime_factors
    for index in range(len(x_naughts)):
        # Getting our respective x_0 and prime factor
        x = x_naughts[index]
        prime = prime_factors[index]
        # Computing N to be our phi value divided by our prime factor
        N = phi//prime
        # Computing the modular inverse of phi/prime and prime mod prime
        inverse = gcdex(N, prime)
        # Adding the modular inverse * phi/prime * x_0
        unmod += inverse*N*x
    # Returning the sum of these modded with our phi value to get our appropriate exponent, as exponents work mod phi
    return unmod%phi
# Setting all our constants     
g = 6
p = 176027859962212193686395560963
A = 20800698928339426549629799773
c1 = 95227620837509305858765921398
c2 = 77039087099389375582873761352
prime = primes(p-1)
a = pohlig(g, A, p)
print("a = "+ str(a))
b = pohlig(g, c1, p)
print("b = " + str(b))
g_ab = faster(g, (a*b), p)
inverse = gcdex(g_ab, p)
m = (inverse * c2)%p
print("The message recovered is " + str(m))

a = 1234567890123456789012345
b = 3333333333333333333333333
The message recovered is 1010101010101010101010101


# Problem 3

How did I find the prime $p$ in such a way that any of the problems on this assignment are possible to do?  Find another such prime $p$, about the same size (plus or minus a few digits). 

In [1]:
import random
import math
# This function is the standard fast factoring calculator that was implemented on the last assignment
def faster(a,p, mod):
    square = a
    answer = 1
    while p > 0:
        if p % 2 == 1:
            p -= 1
            answer *= square
            answer %= mod
        p//=2
        square = square**2 % mod
    return answer % mod
# This function implement the miller-rabin test. It starts by taking in the number to determine if it is a prime.
# It then computes a random a from the range 0->p-1. It first checks is a^p-1 is not 1 and returns false if it is not.
# It then divides p-1 by 2 as many times as needed and stores the multiplicity in a variable k.
# It then iterates of range(k+1) or 0->k and stores the value of a^(2^i*q) in an array. If a^(2^i*q) == 1, it checks
# if the value prior was not a 1 or a -1. If it wasn't it returns false. If the entire while loop executes without 
# breaks, true is returned.
def miller(p):
    prime = p
    p_1 = p-1
    for i in range(1000):
        rand = math.floor((random.random() * p))
        if faster(rand, p_1, p) != 1:
            return False
        k = 0
        while p%2 == 0:
            k += 1
            p/=2
        array = []
        for i in range(k+1):
            modded = faster(rand, (2**k*p), prime)
            array.append(modded)
            if modded == 1:
                if array[i-1] != 1 or array[i-1] != -1:
                    return False
    return True
# This functions computes all primes up to upper. It does so by creating an array and testing if each index in the 
# range is divisible by one of the primes in the array. If so, it breaks from the loop and if not it adds the index
# to the list of primes.
def sieve(upper):
    primes = []
    for index in range(upper+1):
        if index == 0 or index == 1:
            continue
        truth = 1
        for prime in primes:
            if index%prime == 0:
                truth = 0
                break
        if truth:
            primes.append(index)
    return primes   
# This function generates a random potential prime by picking 16 random elements from the list it is given.
# I chose to give it all primes up to 200 so that the resulting number would have a magnitude similar to the prime
# given to us. It then returns prime + 1
def gen_p(primes):
    p = 1
    for i in range(16):
        p *= random.choice(primes)
    return p+1
# This test generates a weak prime from a list of primes given. It iterates over a while loop tied to a truth variable
# Each iteration, a new potential prime is generated and the miller rabin test is run on this number to determine if
# it is prime. If so, it breaks from the loop and returns the number. Otherwise, it keeps iterating until it finds one.
def weak_prime(primes):
    truth = 1
    while truth:
        p_1 = gen_p(primes)
        if miller(p_1):
            truth = 0
    return p_1
primes = sieve(200)
x = weak_prime(primes)
print("The number " + str(x) + " is a weak prime!")


The number 64870226091479967666052824431 is a weak prime!


# Notes

**Note 1** when writing up these assignments, please clearly explain what all the code does.  In particular, please don't make me or the grader run your code, or even look at it.  Before each code block, say what your code does; after the code runs, interpret the results.

**Note 2** You'll need to reuse some of your old code, from assignments 0 and 1.

**Note 3** If your code is taking a long time to run, you're doing something wrong.  I designed and tested this whole assignment on a 10-year-old PC, and everything took less than a second to run.

**Note 4** You might find it helpful to look up the [prime number theorem](https://en.wikipedia.org/wiki/Prime_number_theorem), if you want to know the chances of finding a prime of a given size by sheer random chance.

**Note 5** Feel free to try making a smaller version of these problems for testing - say, with $p = 97$ or whatever.  For primes of that size, brute-forcing things is totally possible.  For the prime I chose, though, not so much.